In [7]:
# Import relevant functions
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import AnOxPePred_funcs as AOf
from clr import *
import numpy as np, pandas as pd, tensorflow as tf
import matplotlib.pyplot as plt
from importlib import reload

In [8]:
# Load data
nick_name = 'Test'
home_path = ''
data_path = os.path.join(home_path, 'Data')
result_path = os.path.join(home_path, 'Result')
embedding_file = os.path.join(data_path, 'Sparse_encoding_v2.txt')
fasta_input = os.path.join(data_path, 'Interesting_peptides.fsa') #np.array(['GIKGIIPAIILEFLEGQLQEVDNNKDAR','LNIQFNIPTPKLC'])

In [19]:
data_df = pd.DataFrame(AOf.get_fsa_file(fasta_input), columns=['Name','Sequence'])

In [17]:
AOf.data_augmentation(data_df.Sequence, embedding_file).shape

(67, 30, 20)

## Run prediction

In [32]:
AnOxPePred_model = 'model'
tf.keras.backend.clear_session()
del AnOxPePred_model

# Load in the model and initiate it with fit (weights have to be initiated before you can load other weights on top)
AnOxPePred_model = AOf.create_AnOxPePred_v1({'y_out': 2})
AnOxPePred_model.fit(np.ones([2,30,5]), np.ones([2,2]), epochs=1 ,verbose=0)

# Load in the correct weights from a trained model
AnOxPePred_model.load_weights(os.path.join(result_path,'AnOxPePred_v1'))

array([[0.04635516, 0.00076723],
       [0.68804353, 0.00743884]], dtype=float32)

In [71]:
b_array = np.array(['LNIQFNIPTPKLC', 'CPFSSDDQFCLKVGV', 'FCLKVGVVHQNGKRRLALVKDNP','FCLKVGVIHQNGKRRLALVK', 'SSDDQFCLKVGVV',
                    'KKPVSKDSPETYEEALKRFAKLLSDRKKL', 'DSPETYEEALKRFAKLLSD', 'FDVIGGTSTGGLLTAMITTPNENNRP',  
                    'GIKGIIPAIILEFLEGQLQEVDNNKDAR'])

In [74]:
a_df = data_df.copy()
augmented_pep = AOf.data_augmentation(b_array, embedding_file)
a_array = AnOxPePred_model.predict(augmented_pep)
#a_df[['FRS', 'Chelator']] = pd.DataFrame(a_array, index=a_df.index)
#a_df.sort_values(by='FRS', ascending=False)
pd.concat([pd.DataFrame(b_array),pd.DataFrame(a_array,columns=['FRS','Chel'])], axis=1)

,0,FRS,Chel
0,LNIQFNIPTPKLC,0.688044,0.007439
1,CPFSSDDQFCLKVGV,0.141870,0.000306
2,FCLKVGVVHQNGKRRLALVKDNP,0.068368,0.004472
3,FCLKVGVIHQNGKRRLALVK,0.124671,0.001212
4,SSDDQFCLKVGVV,0.481137,0.002579
5,KKPVSKDSPETYEEALKRFAKLLSDRKKL,0.006618,0.000124
6,DSPETYEEALKRFAKLLSD,0.018130,0.000349
7,FDVIGGTSTGGLLTAMITTPNENNRP,0.089828,0.000726
8,GIKGIIPAIILEFLEGQLQEVDNNKDAR,0.046355,0.000767
